<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_6_%E1%84%86%E1%85%A5%E1%86%AF%E1%84%90%E1%85%B5_%E1%84%87%E1%85%A6%E1%86%A8%E1%84%90%E1%85%A5_%E1%84%80%E1%85%A5%E1%86%B7%E1%84%89%E1%85%A2%E1%86%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 멀티 벡터 검색 (Multi-Vector Retrieval) 실습

**LangChain 1.0+ | GPT-4o-mini | 네이버 뉴스 데이터셋**

---

## 💡 멀티 벡터 검색이란?

```
일반 검색의 문제:
긴 문서 전체 → 벡터화 → 검색
❌ 긴 문서는 임베딩 시 정보 손실 발생

멀티 벡터 검색의 해결:
요약/작은청크 → 벡터화 (검색용) 정확한 검색
원본/큰청크 → 저장 (반환용) 풍부한 컨텍스트

결과: 검색 정확도 ↑ + LLM 답변 품질 ↑
```

---

# 환경 설정

---

In [ ]:
# LangChain 1.0+ 및 필요 라이브러리 설치
!pip install -U \
    langchain \
    langchain-openai \
    langchain-core \
    langchain-community \
    langchain-chroma \
    langchain-text-splitters \
    chromadb \
    openai \
    datasets \
    tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.7 MB/s eta 0

In [ ]:
# 버전 확인
import langchain
print(f"LangChain 버전: {langchain.__version__}")
print("LangChain 1.0+ 설치 완료!" if langchain.__version__ >= "1.0" else "버전 확인 필요")

LangChain 버전: 1.1.0
LangChain 1.0+ 설치 완료!


In [ ]:
import os
import uuid
from getpass import getpass
from datasets import load_dataset

# LangChain 1.0+ imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.stores import InMemoryStore
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API 키를 입력: ")
print("API 키 설정 완료")

OpenAI API 키를 입력: ··········
API 키 설정 완료


---

# 실습 - 요약으로 검색 → 원본 반환

---

## 📊 시스템 구조

```
┌─────────────────┐
│  사용자 쿼리    │  "경제 뉴스 찾아줘"
└────────┬────────┘
         │
         ▼
┌─────────────────────┐
│  Vector Store       │  요약으로 검색 (빠르고 정확)
│  "한은, 금리 동결"  │  
└────────┬────────────┘
         │ doc_id로 연결
         ▼
┌─────────────────────┐
│  Doc Store          │  원본 반환 (풍부한 컨텍스트)
│  "한국은행은..."    │  
└────────┬────────────┘
         │
         ▼
┌─────────────────┐
│  GPT-4o-mini    │  답변 생성
└─────────────────┘
```

## 데이터 로드

In [ ]:
# 데이터셋 로드
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train")

# 실습용 샘플 (30개)
NUM_SAMPLES = 30
data = dataset.select(range(NUM_SAMPLES))

print(f'{len(data)}개 뉴스 기사 로드 완료\n')
print("="*80)

# 데이터 구조 확인
sample = data[0]
print(f"[샘플 데이터 구조]")
print(f"제목: {sample.get('title', 'N/A')}")
print(f"요약: ({len(sample['summary'])}자):")
print(f"{sample.get('summary', 'N/A')}")
print(f"원문: ({len(sample['document'])}자):")
print(f"{sample.get('document', 'N/A')}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

30개 뉴스 기사 로드 완료

[샘플 데이터 구조]
제목: 추경호 중기 수출지원 총력 무역금융 40조 확대
요약: (170자):
올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했다.
원문: (831자):
앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 

## 멀티 벡터 시스템 초기화

In [ ]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print("임베딩 모델: text-embedding-3-small")

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("LLM: gpt-4o-mini")

# Vectore Store(요약 저장)
vectorstore = Chroma(
    collection_name="news_summaries",
    embedding_function=embeddings
)
print("Vector Store: Chroma")

# Doc Store(원본 저장)
docstore= InMemoryStore()
print("Doc Store: InMemoryStore")

임베딩 모델: text-embedding-3-small
LLM: gpt-4o-mini
Vector Store: Chroma
Doc Store: InMemoryStore


In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# -------------------------------------------------------
# 1) parent 문서 저장 (docstore 역할)
# -------------------------------------------------------
# 예: docstore = { "A": Document(...), "B": Document(...) }
parent_store = docstore

# -------------------------------------------------------
# 2) MultiVectorRetriever 기능 구현
#    - child 벡터 저장소 = vectorstore
#    - parent 문서 = parent_store
#    - id_key = "doc_id"
# -------------------------------------------------------
def multivector_retrieve(query):
    # 1. child 문서 검색
    child_docs = vectorstore.similarity_search(query, k=5)

    # 2. child -> parent id 매핑
    parent_ids = {doc.metadata["doc_id"] for doc in child_docs}

    # 3. parent 문서 반환
    parent_docs = [parent_store[id] for id in parent_ids in id in parent_store]

    return parent_docs

# LCEL Runnable 형태의 retriever 만들기 (MultiVectorRetriever 대체)
retriever = RunnableLambda(multivector_retrieve)

print("Custom MultiVectorRetriever 생성 완료")
print("="*80)

Custom MultiVectorRetriever 생성 완료


## 문서 인덱싱 (요약 → 원본 매핑)

In [ ]:
import uuid
from langchain_core.documents import Document

doc_ids = []
summary_docs = []
full_docs = []

for idx, item in enumerate(data):
    # 고유 ID 생성
    doc_id = str(uuid.uuid4())
    doc_ids.append(doc_id)

    # -------------------------
    # 1) 원본 문서 (DocStore 저장용)
    # -------------------------
    full_doc = Document(
        page_content=item['document'],
        metadata={
            "doc_id": doc_id,
            "title": item.get('title', f'뉴스_{idx}'),
            "type": "full_document"
        }
    )
    full_docs.append(full_doc)

    # -------------------------
    # 2) 요약 문서 (VectorStore 검색용)
    # -------------------------
    summary_doc = Document(
        page_content=item['summary'],
        metadata={"doc_id": doc_id}
    )
    summary_docs.append(summary_doc)

print("\n 문서 인덱싱")
print("="*80)


 문서 인덱싱


In [ ]:
# ---------------------------------------------------------
# STEP 1: Vector Store에 요약 문서 추가
# ---------------------------------------------------------
print("Step 1: Vector Store에 요약 벡터 추가")
vectorstore.add_documents(summary_docs)

# ---------------------------------------------------------
# STEP 2: DocStore에 원본 문서 저장
# ---------------------------------------------------------
print("Step 2: Doc Store에 원본 문서 저장")
docstore.mset(list(zip(doc_ids, full_docs)))

print("="*80)
print("인덱싱 완료! 검색 가능")
print("="*80)

Step 1: Vector Store에 요약 벡터 추가
Step 2: Doc Store에 원본 문서 저장
인덱싱 완료! 검색 가능


## 멀티 벡터 검색 테스트

In [ ]:
from langchain_core.runnables import RunnableLambda

def multivector_retrieve(params: dict):
    query = params["query"]
    k = params.get("k", 3)

    # 1 요약(Child) 문서 검색
    child_docs = vectorstore.similarity_search(query, k=k)

    # 2 child -> parent id 매핑
    parent_ids = {d.metadata["doc_id"] for d in child_docs}
    parent_ids_list = list(parent_ids)

    # 3 parent 문서 조회
    parent_docs = docstore.mget(parent_ids_list)
    parent_docs = [d for d in parent_docs if d is not None]

    return parent_docs

retriever = RunnableLambda(multivector_retrieve)
print("Custom Multi-Vector Retrieve 준비 완료")

Custom Multi-Vector Retrieve 준비 완료


In [ ]:
def search_documents(query, k=3):
    print(f"검색 쿼리: {query}")
    print("="*80)

    # dict 하나를 입력으로 전달
    results = retriever.invoke({"query":query, "k":k})

    print(f"검색 결과: {len(results)}개 문서")

    for i, doc in enumerate(results, 1):
        print(f"\n[문서 {i}]")
        print(f"제목: {doc.metadata.get('title', 'N/A')[:70]}")
        print(f"타입: {doc.metadata.get('type', 'N/A')}")
        print(f"길이: {len(doc.page_content)}자")
        print(f"내용 미리보기:")
        print(f"{doc.page_content[:200]}...")

    print("="*80)
    return results

print("검색 함수 준비 완료")

검색 함수 준비 완료


In [ ]:
# 테스트 1: 경제 관련 검색
results1 = search_documents("경제 성장률과 금리 정책", k=3)

검색 쿼리: 경제 성장률과 금리 정책
검색 결과: 3개 문서

[문서 1]
제목: 추경호 중기 수출지원 총력 무역금융 40조 확대
타입: full_document
길이: 831자
내용 미리보기:
앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 ...

[문서 2]
제목: 글로벌 비즈 가트너 올해 전세계 스마트폰 판매량 7% 감소 전망
타입: full_document
길이: 1796자
내용 미리보기:
경제와이드 모닝벨 글로벌 비즈 임선우 외신캐스터 글로벌 비즈입니다. ◇ 올해 스마트폰 판매 감소 올해 전세계 스마트폰 판매량이 크게 줄어들 것이란 전망이 나왔습니다. 시장조사업체 가트너는 글로벌 스마트폰 판매가 7% 하락할 것으로 내다봤는데요. 경제 전반에 걸친 침체 우려와 중국의 봉쇄조치 여파 그리고 인플레이션으로 소비자들이 지갑을 열기 주저하면서 수요가...

[문서 3]
제목: 전자·철강·석유 업종 하반기 수출감소 불가피
타입: full_document
길이: 1303자
내용 미리보기:
원자재 가격 상승에 따른 수출경쟁력 약화·공급망 애로 타격 글로벌 원자재 수급난 및 공급망 차질로 인해 올해 하반기 우리나라 수출 증가세가 크게 꺾일 것이라는 전망이 제기됐다. 1일 전국경제인연합회가 시장조사 전문기관 모노리서치에 의뢰해 매출액 1000대 기업 중 12대 수출 주력 업종을 대상으로 2022 하반기 수출 전망 조사 를 진행한 결과 기업들은 평...


In [ ]:
# 테스트 2: 기술 관련 검색
results2 = search_documents("인공지능과 기술 혁신", k=3)

검색 쿼리: 인공지능과 기술 혁신
검색 결과: 3개 문서

[문서 1]
제목: 야놀자 포커스미디어와 ‘동네가게 오래함께’ 캠페인 진행
타입: full_document
길이: 522자
내용 미리보기:
사진 야놀자 제공 야놀자가 포커스미디어와 동네가게 오래함께 캠페인을 진행한다고 4일 밝혔다. 이번 캠페인은 지역 내 우수 소상공인을 발굴하고 이들을 위한 맞춤형 광고를 제작해 해당 지역 내 홍보를 지원한다. 총 14억 원 규모의 광고 제작 및 송출 비용은 양사가 전액 부담한다. 야놀자는 제휴점을 대상으로 사연을 공모하고 상권 빅데이터 분석을 진행해 지원 대...

[문서 2]
제목: 과기정통부 7월 한 달간 제11회 정보보호의 달 운영
타입: full_document
길이: 1512자
내용 미리보기:
국정원·행안부·국방부와 협력해 다양한 온·오프라인 행사 진행 과학기술정보통신부와 한국정보보호산업협회 KISIA 은 튼튼한 사이버안보 안전한 디지털강국 를 주제로 7월 한 달 동안 제11회 정보보호의 달을 운영한다고 3일 밝혔다. 정보보호의 달은 증가하는 사이버위협에 대응하여 국민들의 보안 인식을 제고하고 정보보호 실천문화를 확산시키기 위해 매년 7월 운영되...

[문서 3]
제목: SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차
타입: full_document
길이: 983자
내용 미리보기:
SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사이언스는 기존 해외사업개발실을 BD Business Development 1 3실로 확대 재편하고 글로벌 규제 및 허가 전담 조직인 Global RA Regulatory Affairs 실을 신설한다고 1일 밝혔다. SK바이오사이언스는 지난해 코로나19 COVID 19 백신 위...


## RAG 시스템 구축

In [ ]:
# RAG 프롬프트 템플릿
rag_prompt = ChatPromptTemplate.from_template(
    """당신은 한국 뉴스 분석 전문가입니다. 주어진 뉴스 기사를 바탕으로 질문에 답변하세요.

검색된 뉴스 기사:
{context}

질문: {question}

답변 (검색된 기사 내용을 바탕으로 정확하게 답변해주세요):"""
)

# RAG 체인
rag_chain = rag_prompt | llm | StrOutputParser()

print("RAG 시스템 준비")

RAG 시스템 준비


In [ ]:
def ask_question(question, k=3):
    print("질문: {question}")
    print("="*80)

    # 문서 검색
    docs = retriever.invoke({"query": question, "k":k})
    print(f"검색 완료")

    if not docs:
        print("관련 문서를 찾지 못했습니다")
        return None

    for i, doc in enumerate(docs, 1):
        print(f"  {i}. {doc.metadata.get('title', 'N/A')[:60]}")

    # 컨텍스트 생성
    context = "\n".join([
        f"[기사 {i+1}] {doc.metadata.get('title', '')}\n{doc.page_content}"
        for i, doc in enumerate(docs)
    ])

    # 답변 생성
    print(f"\n GPT-4o-mini 답변 생성중")
    answer = rag_chain.invoke({"context": context, "question": question})


    print("\n" + "="*80)
    print(" 답변:")
    print(answer)
    print("="*80)

    return answer

print("RAG 질의응답 함수 준비 완료")

RAG 질의응답 함수 준비 완료


In [ ]:
# RAG 테스트 1
answer1 = ask_question("최근 경제 동향에 대해 설명해주세요")

질문: {question}
검색 완료
  1. “1분기 가계 예금은 늘고 대출 증가세는 둔화”
  2. 전자·철강·석유 업종 하반기 수출감소 불가피
  3. 가뭄 영향에... 금배추 됐네

 GPT-4o-mini 답변 생성중

 답변:
최근 경제 동향은 여러 가지 측면에서 나타나고 있습니다. 

첫째, 가계의 예금이 증가하고 대출 증가세가 둔화되고 있습니다. 한국은행의 조사에 따르면, 올해 1분기 가계와 비영리단체의 순자금운용은 60조 4천억 원으로, 1년 전보다 9조 3천억 원 증가했습니다. 이는 코로나19 지원금 등으로 소득이 늘어나고, 주택매매와 같은 부동산 투자가 줄어들면서 가계의 자금 운용 규모가 확대된 것으로 분석됩니다. 그러나 주식투자 규모는 지난해 1분기보다 증가세가 둔화된 16조 원에 그쳤습니다.

둘째, 하반기 수출 전망이 부정적입니다. 원자재 가격 상승과 공급망 차질로 인해 올해 하반기 우리나라의 수출 증가세가 크게 꺾일 것으로 예상되고 있습니다. 전국경제인연합회가 실시한 조사에 따르면, 기업들은 하반기 수출이 전년 동기 대비 0.5% 증가에 그칠 것으로 전망하고 있으며, 전기전자, 철강, 석유화학 업종은 수출이 감소할 것이라고 응답했습니다. 원자재 가격 상승과 물류비 상승이 주요 요인으로 지목되고 있습니다.

셋째, 농업 부문에서는 가뭄의 영향으로 배추 가격이 급등하고 있습니다. 서울의 대형마트에서 배추 도매가격이 10kg 기준으로 1만500원에 이를 것으로 예상되며, 이는 작년 대비 90.9%, 평년 대비 39% 상승한 수치입니다. 이는 가뭄 등 날씨 영향으로 작황이 부진하고 재배면적이 감소했기 때문입니다.

종합적으로, 최근 경제 동향은 가계의 자금 운용 변화, 수출 감소 전망, 농산물 가격 상승 등 다양한 요소가 복합적으로 작용하고 있는 상황입니다.


In [ ]:
# RAG 테스트 2
answer2 = ask_question("기술 발전이 사회에 미치는 영향은?")

질문: {question}
검색 완료
  1. 윤종규 KB금융 회장 고객의 방파제 역할 수행하는 것이 금융사 핵심
  2. 과기정통부 7월 한 달간 제11회 정보보호의 달 운영
  3. “고객님 소비패턴 보니… 카페할인 큰 카드 알려드리죠”

 GPT-4o-mini 답변 생성중

 답변:
기술 발전은 사회에 여러 가지 긍정적인 영향을 미치고 있습니다. 특히, 데이터 기반의 맞춤형 서비스 제공이 가능해짐에 따라 개인의 소비 패턴과 금융 정보를 분석하여 보다 효율적이고 개인화된 서비스를 제공할 수 있게 되었습니다. 예를 들어, 통신사의 마이데이터 서비스는 고객의 금융정보를 분석하여 최적의 금융 상품이나 서비스를 추천하는 방식으로, 소비자에게 더 나은 선택을 할 수 있도록 돕고 있습니다. 

또한, 기술 발전은 사이버 보안 분야에서도 중요한 역할을 하고 있습니다. 정보보호의 달과 같은 캠페인을 통해 사이버 위협에 대한 인식을 높이고, 정보보호 실천 문화를 확산시키는 데 기여하고 있습니다. 이는 디지털 사회에서 안전한 환경을 조성하는 데 필수적입니다.

결론적으로, 기술 발전은 개인화된 서비스 제공과 사이버 보안 강화 등 다양한 측면에서 사회에 긍정적인 영향을 미치고 있으며, 이는 기업과 소비자 간의 관계를 더욱 발전시키고 있습니다.
